# Force Merge操作
----
ディスクの使用量を削減するために、強制的な Merge操作によってセグメント数を減らします。


**<font color="red">このデモ環境では実施できません。クラスタを運用した場合のイメージとしてお読みください。</font>**

ElasticsearchのDELETE APIを使用したレコードの削除処理は、削除対象レコードに対して削除フラグを設定し、検索対象から除外されるだけとなります。  
そのため実データは残留し、リソースを使用したままになります。

Force Merge APIは、削除フラグが設定されたレコードの実データを完全削除し、インデックスの最適化を行い、シャードのセグメント数を削減します。  
これにより、ディスク使用量を削減することができます。  

付随して、完全削除されたドキュメントの持っていたドキュメントIDも解放されます。

詳細はElasticsearch Referenceの[Force Merge](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/indices-forcemerge.html)を参照してください。  

## 準備
本章のコマンドを実行するための設定を行います。  

セットアップ済みの環境のうちコマンドを送る先であるCoordinating(Client) Nodeのホストアドレスを、次のセルに記入して実行し、保存してください。

In [1]:
%env ES_HOST=XXX.XXX.XXX.231:9200

env: ES_HOST=XXX.XXX.XXX.231:9200


また、データを蓄積する先のインデックスとタイプを、次のセルを実行して保存してください。  

In [2]:
%env INDEX=meteorological-data-*

env: INDEX=meteorological-data-*


## 削除可能なデータ件数の確認
削除フラグがonの状態のデータが何件あるか、Indices Statsで確認します。

次のコマンドを発行してください。  
コマンドの詳細はElasticsearch Referenceの[Indices Stats](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/indices-stats.html)を参照してください。

In [3]:
%%bash
curl -XGET "http://$ES_HOST/$INDEX/_stats?pretty"

{
  "_shards" : {
    "total" : 140,
    "successful" : 140,
    "failed" : 0
  },
  "_all" : {
    "primaries" : {
      "docs" : {
        "count" : 365,
        "deleted" : 0
      },
      "store" : {
        "size_in_bytes" : 860512,
        "throttle_time_in_millis" : 0
      },
      "indexing" : {
        "index_total" : 365,
        "index_time_in_millis" : 8281,
        "index_current" : 0,
        "index_failed" : 0,
        "delete_total" : 0,
        "delete_time_in_millis" : 0,
        "delete_current" : 0,
        "noop_update_total" : 0,
        "is_throttled" : false,
        "throttle_time_in_millis" : 0
      },
      "get" : {
        "total" : 0,
        "time_in_millis" : 0,
        "exists_total" : 0,
        "exists_time_in_millis" : 0,
        "missing_total" : 0,
        "missing_time_in_millis" : 0,
        "current" : 0
      },
      "search" : {
        "open_contexts" : 0,
        "query_total" : 36,
        "query_time_in_millis" : 173,
        "query_cu

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   99k  100   99k    0     0  1321k      0 --:--:-- --:--:-- --:--:-- 1326k


"deleted"に表示されている件数が削除フラグが設定されているドキュメントの数（＝ディスク上から削減可能なデータの数）になります。

## Force Merge APIの発行
実際にAPIを発行します。

次のコマンドを実行してください。

In [4]:
%%bash
curl -XPOST "http://$ES_HOST/$INDEX/_forcemerge?pretty"

{
  "_shards" : {
    "total" : 140,
    "successful" : 140,
    "failed" : 0
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    84  100    84    0     0    404      0 --:--:-- --:--:-- --:--:--   405
